In [ ]:
import h5py, sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
sys.path.append('/mnt/cbis/home/benedikt/.local/dragonfly/utils/py_src')

In [ ]:
import writeemc
import detector
import reademc

In [ ]:
path_to_data = "../../data/sparse/" # scratch/sparse on Maxwell
path_to_aux = "../../data/aux/" # scratch/benedikt/aux on Maxwell
path_to_geometry = "../../geometry/"
path_to_recons = "../../data/recons/"

## Load EMC reconstruction

In [ ]:
rr = 1

In [ ]:
emc_folder = path_to_recons + "melv_0001/"
emc_output = emc_folder + "data/r%d/output_060.h5" %rr
emc_photons = emc_folder + 'photons.txt'
emc_detector = path_to_recons + "det/det_lowq8.h5"

In [ ]:
with h5py.File(emc_output,'r') as f:
    occupancies = f['occupancies'][:]
    likelihood = f['likelihood'][:]
    orientations = f['orientations'][:]
    images = f['intens'][:]

## Determine modes

In [ ]:
nr_rot = 180
nr_static = 5
nr_modes = occupancies.shape[1] - nr_static
modes = orientations % (nr_modes)
rotind = orientations // (nr_modes)
modes[rotind >= nr_rot] = orientations[rotind >= nr_rot] - nr_modes * (nr_rot - 1)
blacklisted = (occupancies == 0).all(axis=1)
ndata = len(modes) - blacklisted.sum()

## Inspect 2D classes

In [ ]:
N = occupancies.shape[1]
ncols = 8
nrows = N//ncols
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4,nrows*4), dpi=200)
for i in range(N):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
    nclass = ((modes==i) & (~blacklisted)).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[i], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %i, color='r')
plt.savefig("../../plots/2dclasses.png", bbox_inches='tight')
plt.show()

## Inspect static classes

In [ ]:
fig, axes = plt.subplots(ncols=nr_static, nrows=1, figsize=(nr_static*4,4), dpi=100)
[ax.axis('off') for ax in axes]
for i in range(nr_static):
    nclass = ((modes==nr_modes+i) & (~blacklisted)).sum()
    axes[i].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[i].text(0,0, "%d" %(nr_modes+i), color='r')
    axes[i].imshow(images[-nr_static+i], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
plt.show()

## Inspect ring classes

In [ ]:
rings = 4,7,16
fig, axes = plt.subplots(ncols=len(rings), nrows=1, figsize=(len(rings)*4,4), dpi=100)
[ax.axis('off') for ax in axes]
for i in range(len(rings)):
    nclass = ((modes==rings[i]) & (~blacklisted)).sum()
    axes[i].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[i].text(0,0, "%d" %(rings[i]), color='r')
    axes[i].imshow(images[rings[i]], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
plt.show()

## Inspect doubles classes

In [ ]:
doubles = 10,11,12,13,21,25,26,27,29,34
N = len(doubles)
ncols = 5
nrows = int(np.ceil(N/ncols))
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*4,nrows*4), dpi=100)
for i in range(ncols*nrows):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
for i in range(N):
    c,r = i//ncols, i%ncols
    nclass = ((modes==doubles[i]) & (~blacklisted)).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[doubles[i]], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %doubles[i], color='r')
plt.show()

## Inspect other strange classes

In [ ]:
strange = 1,2,3,5,14,15,28,30,31
N = len(strange)
ncols = 5
nrows = int(np.ceil(N/ncols))
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*4,nrows*4), dpi=100)
for i in range(ncols*nrows):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
for i in range(N):
    c,r = i//ncols, i%ncols
    nclass = ((modes==strange[i]) & (~blacklisted)).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[strange[i]], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %strange[i], color='r')
plt.show()

## Classes to be removed/kept

In [ ]:
remove = list(range(nr_modes, nr_modes+nr_static)) + list(rings) + list(doubles) + list(strange)
keep = list(range(nr_modes + nr_static))
for r in remove: keep.remove(r)

## Inspect classes to keep

In [ ]:
N = len(keep)
ncols = 5
nrows = int(np.ceil(N/ncols))
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*4,nrows*4), dpi=100)
keeptotal = 0
for i in range(ncols*nrows):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
for i in range(N):
    c,r = i//ncols, i%ncols
    nclass = ((modes==keep[i]) & (~blacklisted)).sum()
    keeptotal += nclass
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[keep[i]], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %keep[i], color='r')
plt.show()

In [ ]:
print("Keeping %d frames for 3D [%.2f%%]" %(keeptotal, keeptotal/ndata*100))

## Write selection into blacklist file

In [ ]:
blacklist = np.zeros(occupancies.shape[0], dtype=int)
for m in remove:
    blacklist[np.where(modes == m)[0]] = 1
ndata = blacklist.shape[0] - blacklist.sum()

In [ ]:
np.savetxt(emc_folder + "blacklist_%d_%d.dat" %(rr,ndata), blacklist, fmt='%d')

## Select for ring features only

In [ ]:
remove = list(range(nr_modes+1))
remove.remove(9)

In [ ]:
blacklist = np.zeros(occupancies.shape[0], dtype=int)
for m in remove:
    blacklist[np.where(modes == m)[0]] = 1
ndata = blacklist.shape[0] - blacklist.sum()

In [ ]:
np.savetxt(path_to_recons + 'ring_0000/' + "blacklist_%d_%d.dat" %(rr,ndata), blacklist, fmt='%d')